In [16]:
import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

from datetime import datetime
import random

from typing import TypedDict, Annotated, List, Literal, Dict
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_agent
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_tavily import TavilySearch
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, tools_condition

from utils.utils import remove_think_content_for_qwen

In [2]:
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [5]:
llm = ChatGroq(model="qwen/qwen3-32b")

### Define state

In [14]:
class OrchestratorState(MessagesState):
    """State for the multi-agent system"""
    next_agent: str = ""
    data_retrieved: str = ""
    eligibility_check: bool = False
    content: str = ""
    communication: str = ""
    task_complete: bool = False

### Define Orchestrator Agent

In [9]:
def create_orchestrator_chain():
    """Create the orchestrator decision chain"""

    orchestrator_prompt = ChatPromptTemplate.from_messages([
        ("system", 
        """
        You are an orchestrating command managing a team of agents:
        1. Data Retriever: Fetches patient data from the database
        2. Eligibility Checker: Checks if patient has to be requested for a google review
        3. Content Generator: Develops a message to be sent to the patient
        4. Communication: Sends the message/email to the patient

        Based on the current state and conversation, decide which agent should work next.
        If the task is complete, respond with 'DONE'.

        Current state:
        - Has data: {has_data}
        - Has eligibility: {has_eligibility}
        - Has content: {has_content}
        - Message sent: {has_message}

        Respond with ONLY the agent name (data/eligibility/content/communication) or 'DONE'.
        """)
    ])

    return orchestrator_prompt | llm

In [13]:
decision = create_orchestrator_chain().invoke({
        "has_data": False,
        "has_eligibility": False,
        "has_content": False,
        "has_message": False,
    })

decision_text = remove_think_content_for_qwen(decision.content.strip().lower())
print(decision_text)

data


In [15]:
def orchestrator_agent(state: OrchestratorState) -> Dict:
    """Orchestrator decides next agent using Groq LLM"""
    
    messages = state["messages"]
    
    # Check what's been completed
    has_data = bool(state.get("data_retrieved", ""))
    has_eligibility = bool(state.get("eligibility_check", ""))
    has_content = bool(state.get("content", ""))
    has_message = bool(state.get("communication", ""))
    
    # Get LLM decision
    chain = create_orchestrator_chain()
    decision = chain.invoke({
        "has_data": has_data,
        "has_eligibility": has_eligibility,
        "has_content": has_content,
        "has_message": has_message,
    })
    
    # Parse decision
    decision_text = remove_think_content_for_qwen(decision.content.strip().lower())
    print(decision_text)
    
    # Determine next agent
    if "done" in decision_text or has_message:
        next_agent = "end"
        supervisor_msg = "✅ Supervisor: All tasks complete! Great work team."
    elif "data" in decision_text or not has_data:
        next_agent = "data"
        supervisor_msg = "📋 Supervisor: Let's start with data retrieval. Assigning to Data Retriever..."
    elif "eligibility" in decision_text or (has_data and not has_eligibility):
        next_agent = "eligibility"
        supervisor_msg = "📋 Supervisor: Data Retrieved. Time for eligibility check. Assigning to Eligibility Checker..."
    elif "content" in decision_text or (has_eligibility and not has_content):
        next_agent = "content"
        supervisor_msg = "📋 Supervisor: Eligibility check complete. Let's create the message content. Assigning to Content Generator..."
    elif "communication" in decision_text or (has_content and not has_message):
        next_agent = "content"
        supervisor_msg = "📋 Supervisor: Message content create. Let's send the message. Assigning to Communication..."
    else:
        next_agent = "end"
        supervisor_msg = "✅ Supervisor: Task seems complete."
    
    return {
        "messages": [AIMessage(content=supervisor_msg)],
        "next_agent": next_agent
    }

In [ ]:
# ===================================
# Agent 1: Data Retriever
# ===================================

def data_retrieval_agent(state: OrchestratorState) -> Dict:
    """Data Retriever picks up mock data from a csv"""
    
    # Get research from LLM
    research_response = llm.invoke([HumanMessage(content=research_prompt)])
    research_data = research_response.content
    
    # Create agent message
    agent_message = f"🔍 Researcher: I've completed the research on '{task}'.\n\nKey findings:\n{research_data[:500]}..."
    
    return {
        "messages": [AIMessage(content=agent_message)],
        "research_data": research_data,
        "next_agent": "supervisor"
    }